# Open Food Facts — Project Loader & Column Drop (CSV only)

Loads the dataset from `../data/openfoodfacts/en.openfoodfacts.org.products.csv` using a tolerant parser,
drops predefined empty/useless columns, and prints the remaining schema.

In [ ]:
# --- Project-correct path (CSV only) ---
from pathlib import Path

DATA_PATH = (Path.cwd().resolve() / ".." / "data" / "openfoodfacts" / "en.openfoodfacts.org.products.csv")
INPUT_FILE_NAME = str(DATA_PATH)

if not DATA_PATH.exists():
    raise FileNotFoundError(f"Dataset not found: {DATA_PATH}")
print("Resolved path ->", INPUT_FILE_NAME)

In [ ]:
# --- Robust load (python engine, skips malformed rows) ---
import pandas as pd, csv

# Optional LIMIT to load only first N rows for speed; set to None to read all
LIMIT = None
nrows = LIMIT if (isinstance(LIMIT, int) and LIMIT > 0) else None

df = pd.read_csv(
    INPUT_FILE_NAME,
    sep="\t",                 # OFF "csv" is tab-separated
    engine="python",          # tolerant parser for messy lines
    on_bad_lines="skip",      # skip malformed rows
    dtype="string",
    na_values=["", " ", "NA", "N/A", "null", "NULL"],
    keep_default_na=True,
    quoting=csv.QUOTE_MINIMAL,
    escapechar="\\",
    # NOTE: low_memory is NOT supported with python engine; do not pass it
)

print(f"Loaded rows: {len(df):,}; columns: {len(df.columns):,}")
display(df.head(3))

In [ ]:
# --- Drop specified empty/useless columns ---
to_drop = ['additives', 'allergens_en', 'capric-acid_100g', 'carbohydrates-total_100g', 'cities', 'isomalt_100g', 'maltitol_100g', 'methylsulfonylmethane_100g', 'nutrition-score-uk_100g', 'psicose_100g', 'sorbitol_100g', 'caprylic-acid_100g', 'galactose_100g', 'nervonic-acid_100g', 'water-hardness_100g', 'acidity_100g', 'chlorophyl_100g', 'elaidic-acid_100g', 'lauric-acid_100g', 'caproic-acid_100g', 'cerotic-acid_100g', 'gamma-linolenic-acid_100g', 'glycemic-index_100g', 'erucic-acid_100g', 'montanic-acid_100g', 'url', 'creator', 'created_t', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime']
existing = [c for c in to_drop if c in df.columns]
before = len(df.columns)
df = df.drop(columns=existing, errors="ignore")
after = len(df.columns)
print(f"Dropped {len(existing)} columns. Columns before: {before}, after: {after}")
if existing:
    print(", ".join(existing))

In [ ]:
# --- List all remaining column names ---
print("Total columns:", len(df.columns))
for c in df.columns:
    print(c)

In [ ]:
# --- Normalize date formats & columns (safe, regex-enabled) ---
import re
import pandas as pd

if 'df' not in globals():
    raise NameError("DataFrame 'df' is not defined. Run the loader cell first.")

# Strategy: match any column ending with '_datetime' and coerce to ISO UTC.
# You can add more entries (e.g., numeric columns) by extending this list.
strategy = [
    {"colname": r".*_datetime$", "dtype": "datetime", "filler": "1970-01-01T00:00:00Z"},
    # Example numeric normalization (uncomment to use):
    # {"colname": r"^energy_100g$", "dtype": "float", "filler": 0.0},
]

# Ensure all columns are strings before type ops; preserves original text if needed
for col in df.columns:
    try:
        df[col] = df[col].astype("string")
    except Exception:
        pass

# Apply normalization per provided logic
for col in df.columns:
    print(f"colname [{col}]:")
    for s in strategy:
        # Match by regex (supports '*' style via regex pattern in s['colname'])
        if "*" in s["colname"]:
            # For safety: treat '*' as literal wildcard -> convert to regex '.*'
            pattern = s["colname"].replace("*", ".*")
            m = re.search(pattern, col)
            if m is None:
                continue
            print(f"\t[{col}] matches <{s['colname']}>.")
        else:
            m = re.search(s["colname"], col)
            if m is None:
                continue
            print(f"\t[{col}] matches <{s['colname']}>.")

        df_c1 = df[col]
        replace_complete = False

        if s["dtype"] == "datetime":
            print(f"\t[{col}] is converted astype <datetime>.")
            while True:
                try:
                    # Expect ISO UTC 'YYYY-MM-DDTHH:MM:SSZ'
                    df_c1 = pd.to_datetime(df_c1, format="%Y-%m-%dT%H:%M:%SZ")
                    replace_complete = True
                    break
                except ValueError as e:
                    m = re.search(r"time data '([^']*)'", str(e))
                    if m is None:
                        print(f"\t\tUnexpected error string <{str(e)}>")
                        break
                    evil_string = m.group(1).replace("'", "")
                    idx = df[df[col] == evil_string].index
                    df.loc[idx, col] = s["filler"]
                    print(f"\t\tReplace string <{evil_string}>[{len(idx)} rows] with <{s['filler']}>.")
            if replace_complete:
                # Final cast & reformat to canonical ISO UTC
                dt = pd.to_datetime(df_c1, errors="coerce", utc=True)
                df[col] = dt.dt.strftime("%Y-%m-%dT%H:%M:%SZ")

        else:
            print(f"\t[{col}] is converted astype <{s['dtype']}>.")
            while True:
                try:
                    df_c1 = df_c1.astype(s["dtype"], errors="raise")
                    replace_complete = True
                    break
                except ValueError as e:
                    if str(e) == "cannot convert float NaN to integer":
                        idx = df[df[col].isnull()].index
                        df.loc[idx, col] = s["filler"]
                    else:
                        m = re.search(r"'(.*)'", str(e))
                        if m is not None:
                            evil_string = m.group(1).replace("'", "")
                            idx = df[df[col] == evil_string].index
                            df.loc[idx, col] = s["filler"]
                            print(f"\t\tReplace string <{evil_string}>[{len(idx)} rows] with <{s['filler']}>.")
                        else:
                            print(f"\t\tUnexpected error string <{str(e)}>")
                            break
            if replace_complete:
                df[col] = df[col].fillna(s["filler"]).astype(s["dtype"])

print("Normalization complete.")